### HARDWARE OPTIMIZATIONS

1. TensorRT
2. ONNX
3. OpenVINO

### 1. TensorRT

##### export config

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

# TensorRT FP32
out = model.export(format="engine", imgsz=640, dynamic=True, verbose=False, batch=8, workspace=2)

# TensorRT FP16
out = model.export(format="engine", imgsz=640, dynamic=True, verbose=False, batch=8, workspace=2, half=True)

# TensorRT INT8 with calibration `data` (i.e. COCO, ImageNet, or DOTAv1 for appropriate model task)
out = model.export(
    format="engine", imgsz=640, dynamic=True, verbose=False, batch=8, workspace=2, int8=True, data="coco8.yaml"
)

##### inference

In [ ]:
import cv2

from ultralytics import YOLO

model = YOLO("yolov8n.engine")
img = cv2.imread("path/to/image.jpg")

for _ in range(100):
    result = model.predict(
        [img] * 8,  # batch=8 of the same image
        verbose=False,
        device="cuda",
    )

##### validation

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.engine")
results = model.val(
    data="data.yaml",  # COCO, ImageNet, or DOTAv1 for appropriate model task
    batch=1,
    imgsz=640,
    verbose=False,
    device="cuda",
)

### 2. ONNX Runtime

In [2]:
model_path = "yolo11/yolov11_5_x_tuned.pt"

In [3]:
from ultralytics import YOLO

model = YOLO(model_path)

model.export(format="onnx")

Ultralytics 8.3.59 🚀 Python-3.11.11 torch-2.5.1 CPU (Apple M3 Pro)
YOLO11x summary (fused): 464 layers, 56,839,729 parameters, 0 gradients, 194.5 GFLOPs

PyTorch: starting from 'yolo11/yolov11_5_x_tuned.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 15, 8400) (109.1 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.47...
ONNX: export success ✅ 2.2s, saved as 'yolo11/yolov11_5_x_tuned.onnx' (217.1 MB)

Export complete (3.4s)
Results saved to /Users/ayush/Documents/projects/yolov8-inference-optimization/yolo11
Predict:         yolo predict task=detect model=yolo11/yolov11_5_x_tuned.onnx imgsz=640  
Validate:        yolo val task=detect model=yolo11/yolov11_5_x_tuned.onnx imgsz=640 data=./doclaynet.yaml  
Visualize:       https://netron.app


'yolo11/yolov11_5_x_tuned.onnx'

In [4]:
import onnx

onnx_model_path = "yolo11/yolov11_5_x_tuned.onnx"

onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)
